In [ ]:
# Instalar las bibliotecas necesarias
!pip install yahoo_fin
!pip install yahoo_fin --upgrade
!pip install pandas_datareader

# Importar las bibliotecas necesarias
import yahoo_fin as yfin  # Importar la biblioteca yahoo_fin
import yahoo_fin.stock_info as si  # Importar la submódulo stock_info de yahoo_fin
import pandas as pd  # Importar pandas para manejo de datos
import numpy as np  # Importar numpy para operaciones numéricas
from datetime import datetime  # Importar datetime para manejo de fechas
import statistics  # Importar statistics para cálculos estadísticos
import matplotlib.pyplot as plt  # Importar matplotlib para gráficos
from scipy import stats  # Importar scipy para estadísticas avanzadas
import seaborn as sns  # Importar seaborn para gráficos avanzados
from pandas_datareader import data  # Importar data de pandas_datareader para obtener datos financieros

# Obtener datos históricos de las acciones que conforman el portafolio desde Yahoo Finance
IBM_s_t = si.get_data("IBM", start_date="11/28/2018", end_date="11/28/2023")
AMZN_s_t = si.get_data("AMZN", start_date="11/28/2018", end_date="11/28/2023")
AAPL_s_t = si.get_data("AAPL", start_date="11/28/2018", end_date="11/28/2023")
MSFT_s_t = si.get_data("MSFT", start_date="11/28/2018", end_date="11/28/2023")
TSLA_s_t = si.get_data("TSLA", start_date="11/28/2018", end_date="11/28/2023")
ORCL_s_t = si.get_data("ORCL", start_date="11/28/2018", end_date="11/28/2023")
JNJ_s_t = si.get_data("JNJ", start_date="11/28/2018", end_date="11/28/2023")
GM_s_t = si.get_data("GM", start_date="11/28/2018", end_date="11/28/2023")
BABA_s_t = si.get_data("BABA", start_date="11/28/2018", end_date="11/28/2023")

# Crear un DataFrame con los precios de cierre ajustados de cada acción y asignar nombres a las columnas
portafolio = pd.concat([IBM_s_t["adjclose"], AMZN_s_t["adjclose"], AAPL_s_t["adjclose"], MSFT_s_t["adjclose"], TSLA_s_t["adjclose"], ORCL_s_t["adjclose"], JNJ_s_t["adjclose"], GM_s_t["adjclose"], BABA_s_t["adjclose"]], axis=1)
portafolio.columns = ["IBM", "AMZN", "AAPL", "MSFT", "TSLA", "ORCL", "JNJ", "GM", "BABA"]
portafolio.head()  # Mostrar las primeras filas del DataFrame

# Graficar los valores de cierre ajustado de las acciones normalizados a porcentajes
(portafolio / portafolio.iloc[0] * 100).plot(figsize=(10, 10))

# Crear una lista con el nombre de las empresas que conforman el portafolio
list_portafolio = ["IBM", "AMZN", "AAPL", "MSFT", "TSLA", "ORCL", "JNJ", "GM", "BABA"]

# Definir una función para calcular los retornos diarios de las acciones en el portafolio
def calcular_retornos(portafolio):
    return portafolio.pct_change()  # Calcular los retornos porcentuales diarios

retornos = calcular_retornos(portafolio)  # Calcular los retornos
retornos.dropna()  # Eliminar filas con valores NaN
retornos.sum()  # Sumar los retornos

# Mostrar el DataFrame de retornos
retornos

# Definir una función para calcular los retornos logarítmicos
def retornos_log(portafolio):
    return np.log(portafolio) - np.log(portafolio.shift(1))  # Calcular los retornos logarítmicos

log_retornos = retornos_log(portafolio)  # Calcular los retornos logarítmicos
log_retornos = log_retornos.dropna()  # Eliminar filas con valores NaN
log_retornos.sum()  # Sumar los retornos logarítmicos

# Mostrar el DataFrame de retornos logarítmicos
log_retornos

# Graficar los retornos logarítmicos
log_retornos.plot(figsize=(10, 10))

# Asignar pesos aleatorios a las acciones
num_stocks = len(list_portafolio)  # Obtener el número de acciones en el portafolio
random_array = np.random.random(num_stocks)  # Generar un arreglo de números aleatorios

# Normalizar los pesos para que su suma sea 1
pesos = np.random.random(num_stocks)
pesos /= np.sum(pesos)

# Mostrar los pesos asignados
pesos

# Calcular el retorno del portafolio usando los pesos asignados y multiplicarlo por 250 (número aproximado de días de trading en un año)
np.sum(pesos * log_retornos.mean()) * 250

# Calcular la varianza del retorno del portafolio usando el producto punto
varianza = np.dot(pesos.T, np.dot(log_retornos.cov() * 250, pesos))

# Calcular la volatilidad del portafolio tomando la raíz cuadrada de la varianza
volatilidad = np.sqrt(varianza)
volatilidad

# SIMULACIONES
retornos_portafolio = []
volatilidades_portafolio = []

# Realizar 1000 simulaciones para calcular retornos y volatilidades de diferentes combinaciones de pesos
for x in range(1000):
    pesos = np.random.random(num_stocks)
    pesos /= np.sum(pesos)
    retornos_portafolio.append(np.sum(pesos * log_retornos.mean()) * 250)
    volatilidades_portafolio.append(np.sqrt(np.dot(pesos.T, np.dot(log_retornos.cov() * 250, pesos))))

retornos_portafolio = np.array(retornos_portafolio)
volatilidades_portafolio = np.array(volatilidades_portafolio)
portafolios2 = pd.DataFrame({"Retorno": retornos_portafolio, "Volatilidad": volatilidades_portafolio})
portafolios2.tail()  # Mostrar las últimas filas del DataFrame

# Graficar la frontera eficiente comparando el riesgo (volatilidad) contra la ganancia (retorno) del portafolio
portafolios2.plot(x="Volatilidad", y="Retorno", kind="scatter", figsize=(10, 6))
plt.xlabel("Volatilidad")
plt.ylabel("Retorno")

# Calcular los retornos esperados
RetornosE = pd.DataFrame(log_retornos.mean())  # Obtener el promedio de los retornos logarítmicos de cada acción
RetornosE

# Calcular el riesgo individual de cada acción (desviación estándar)
RiesgoI = pd.DataFrame(log_retornos.std())
RiesgoI

# Crear la matriz de varianza y covarianza
datacov = log_retornos.select_dtypes(include=["float64", "int"])  # Seleccionar solo las columnas de tipo float64 y int
corr_matrix = datacov.corr(method="pearson")  # Calcular la matriz de correlación usando el método de Pearson
corr_matrix

# CALCULAR EL RENDIMIENTO ESPERADO DEL PORTAFOLIO
pesos = np.random.random(9)  # Asignar pesos aleatorios a cada acción
pesos /= np.sum(pesos)  # Normalizar los pesos para que su suma sea 1
pesos

RetE = RetornosE.iloc[:, 0].values  # Obtener los valores de los retornos esperados
print(RetE)

np.matmul(pesos, RetE)  # Calcular el rendimiento esperado del portafolio usando el producto punto

# CALCULAR LA VARIANZA DEL PORTAFOLIO
transpesos = pesos.transpose()  # Transponer el vector de pesos
SxW = np.dot(corr_matrix, transpesos)  # Calcular el producto punto de la matriz de correlación y los pesos transpuestos
VARP = np.matmul(pesos, SxW)  # Calcular la varianza del portafolio usando el producto punto
VARP

# CALCULAR EL RIESGO INDIVIDUAL DEL PORTAFOLIO
RIP = np.sqrt(VARP)  # Calcular el riesgo individual del portafolio tomando la raíz cuadrada de la varianza
RIP

# Instalar la paquetería PyPortfolioOpt y las funciones para resolver problemas de frontera eficiente
!pip install PyPortfolioOpt

# Importar la función EfficientFrontier de la biblioteca PyPortfolioOpt
from pypfopt.efficient_frontier import EfficientFrontier

# Usar la función EfficientFrontier para calcular los portafolios óptimos, con la condición de que los pesos deben ser positivos
ef = EfficientFrontier(RetE, corr_matrix, weight_bounds=(0, 1))

# Obtener los pesos del portafolio con la máxima proporción de Sharpe
weights = ef.max_sharpe()
weights

# Limpiar los pesos del portafolio para eliminar valores cercanos a cero
cleaned_weights = ef.clean_weights()
cleaned_weights

# Obtener el rendimiento esperado, la volatilidad y el índice de Sharpe del portafolio
ef.portfolio_performance(verbose=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 3.8 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=3463a04aaaa4b60c018d3a6e12c3bc466ec9ef20a08f7b1a1a168f71c7bb943e
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
